In [1]:
import pymongo
import requests
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs

In [2]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
#NASA Mars News
news_url = 'https://mars.nasa.gov/news/'
browser.visit(news_url)

In [4]:
html = browser.html
nsoup = bs(html, 'html.parser')
lists = nsoup.find_all('li', class_='slide')

In [5]:
html = browser.html
nsoup = bs(html, "html.parser")  
content_title = lists[0].find('div', class_='content_title')
news_title = content_title.text.strip()    
article_teaser_body = lists[0].find('div', class_='article_teaser_body')
news_p = article_teaser_body.text.strip()
print(news_title)
print(news_p)

NASA's Mars Reconnaissance Orbiter Undergoes Memory Update
Other orbiters will continue relaying data from Mars surface missions for a two-week period.


In [6]:
nasa_news_text = []
for row in nsoup.find_all('div',attrs={"class" : "article_teaser_body"}):
    nasa_news_text.append(row)
    
print(nasa_news_text[0].text)

Other orbiters will continue relaying data from Mars surface missions for a two-week period.


In [7]:
#JPL Mars Space Images - Featured Image
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_url)
jpl_html = browser.html
jsoup = bs(jpl_html,'html.parser')

In [8]:
img_link = jsoup.find('div',class_='carousel_container').article.footer.a['data-fancybox-href']
print(img_link)
base_link = jsoup.find('div', class_='jpl_logo').a['href']
print(base_link)

/spaceimages/images/mediumsize/PIA15254_ip.jpg
//www.jpl.nasa.gov/


In [9]:
featured_img_url = base_link.split('/')[2] + img_link
print(featured_img_url)
featured_img_title = jsoup.find('h1', class_="media_feature_title").text.strip()
print(featured_img_title)

www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA15254_ip.jpg
Dusty Space Cloud


In [13]:
#scrape the latest Mars weather tweet from the Mars Weather twitter account
weather_url = 'https://twitter.com/marswxreport?lang=en'
result = requests.get(weather_url)

In [14]:
#make it text
html = result.text

#create BeautifulSoup object and parse
wsoup = bs(html, 'html.parser')

In [15]:
#get the weather from the newest tweet
mars_weather = wsoup.find(class_='tweet-text').get_text()
mars_weather

'InSight sol 437 (2020-02-18) low -94.7ºC (-138.5ºF) high -9.9ºC (14.1ºF)\nwinds from the SSE at 6.2 m/s (13.8 mph) gusting to 21.6 m/s (48.3 mph)\npressure at 6.30 hPapic.twitter.com/SLsDLVefeh'

In [16]:
#mars facts
fact_url = "https://space-facts.com/mars/"
fact_table = pd.read_html(fact_url)
mars_fact_table = fact_table[0]
mars_fact_table.head()

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [17]:
mars_fact_table_html = mars_fact_table.to_html(header=False, index=False)
mars_fact_table_html = mars_fact_table_html.replace('\n', '')
print(mars_fact_table_html)

<table border="1" class="dataframe">  <tbody>    <tr>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>


In [18]:
hem_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hem_url)
hem_html = browser.html
hsoup = bs(hem_html, 'html.parser')

In [19]:
items = [(div.h3.text, div.a['href']) for div in 
        hsoup.find_all('div', class_='description')]
hemisphere_image_urls = []

for title,img_url in items:
    #print(title,img_url)
    hemisphere_dict = {}
    hemisphere_dict['title'] = title
    browser.visit('https://astrogeology.usgs.gov' + img_url)
    #print('https://astrogeology.usgs.gov' + img_url)
    hem_img_html = browser.html
    imgsoup = bs(hem_img_html, 'html.parser')
    image_url = imgsoup.find('img', class_='wide-image')['src']
    hemisphere_dict['img_url'] = 'https://astrogeology.usgs.gov' + image_url
    hemisphere_image_urls.append(hemisphere_dict)
    
print(hemisphere_image_urls)   

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]
